### Goal: Compute vaccine plan for surging countries, priority cases or prevalence 

Prioritize surging countries
- supply: additional 5 million daily for 3 months
- overall supply proportional to chosen feature

procedure
- prepare feature dataframe
- rank countries by feature
- choose the top N countries in ranking
- distribute donation proportional to features

feature options:

- recent case
- recent prevalence
- case in the next 3 months
- prevalence in the next 3 months 

In [18]:
from ABMpy_region.model import Meta_ABM 
from ABMpy_region.agent import ABM
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict 
import os
import sys
import matplotlib.backends.backend_pdf

In [19]:
# region list for simulations
with open('region_list.pkl','rb') as f:
    regions=(pickle.load(f))

In [20]:
# pre pare dates 
start_data=0
end_data=581

from datetime import datetime, timedelta
base = datetime(2020, 1, 1)
arr = np.array([base + timedelta(days=int(i)) for i in range(start_data,end_data+1)])

In [21]:
arr[546]

datetime.datetime(2021, 6, 30, 0, 0)

In [22]:
# input needed
vaccine=pd.read_csv("vaccine_info_83.csv")
case=pd.read_csv("prevalance_84.csv")

# imputation 
vaccine.fillna(0,inplace=True)

- recent cases
- recent prevalence

In [ ]:

# recent prevalence 
# prevelance 15 day average:
p=np.zeros_like(case['520'])
for i in range(532,547):
    p+=case[str(i)]
p=p/15
#p=data['520']

# combine dataframes to "new_cases"
new_cases=case[['location','population']].copy()
new_cases['cases']=case['population']*p # recent cases
new_cases['prevalence_per_million']=p*10**6 # recent prevalence
new_cases=new_cases[new_cases.location.isin(regions+['US'])] # only include regions we simulated 

# combine US state data to US national level 
us_vac=0
us_frac=0
us_state_weight=[]
us_state_cum=[]
for i,row in new_cases.iterrows():
    r=row.location 
    if r=='US': continue
        
    # last month's and onward's total dosage
    vac=vaccine.loc[vaccine.location==r,'dailyVac1_permillion_duringLastMonth'].item()*2*row.population/10**6
    
    new_cases.loc[i,'vaccine_self_546']=vac
    cum=vaccine.loc[vaccine.location==r,'vac1RateCumu_asOfLastMonth'].item()*2/100
    new_cases.loc[i,'vaccine_cum_fraction_516']=cum
                                                           
    if r.startswith('US.'):
        us_vac+=vac
        us_state_weight.append(row.population/10**6)
        us_state_cum.append(cum)
new_cases.loc[new_cases.location=='US','vaccine_self_546']=us_vac 

us_state_weight=np.array(us_state_weight)
us_state_cum=np.array(us_state_cum)

new_cases.loc[new_cases.location=='US','vaccine_cum_fraction_516']=(us_state_weight*us_state_cum).sum()/us_state_weight.sum()
new_cases=new_cases[~new_cases.location.str.startswith('US.')] # remove state level data

# vaccine covarge on 6/30 (7/1 is the start day for projection)
new_cases['vaccine_cum_fraction_546']=new_cases['vaccine_cum_fraction_516']+new_cases.vaccine_self_546*30/new_cases.population

In [23]:
data

,location,population,cases,prevalence_per_million,vaccine_self_546,vaccine_cum_fraction_516,vaccine_cum_fraction_546
0,Afghanistan,3.892834e+07,1713.333333,44.012493,4734.532258,0.007621,0.011269
1,Albania,2.877800e+06,3.533333,1.227790,2956.354839,0.131937,0.162756
2,Algeria,4.385104e+07,365.800000,8.341877,39112.903226,0.000855,0.027614
4,Angola,3.286627e+07,126.133333,3.837775,10134.080645,0.013083,0.022333
5,Antigua and Barbuda,9.792800e+04,0.000000,0.000000,317.758065,0.132453,0.229797
...,...,...,...,...,...,...,...
156,Uzbekistan,3.346920e+07,425.133333,12.702226,30624.161290,0.024541,0.051991
157,Venezuela,2.843594e+07,1184.733333,41.663234,18564.322581,0.005556,0.025142
159,Zambia,1.838396e+07,2594.800000,141.144811,104.209677,0.003988,0.004158
160,Zimbabwe,1.486293e+07,563.733333,37.928823,5168.790323,0.033396,0.043829


- case in the next 3 months
- prevalence in the next 3 months 

In [24]:
# prepare for other features

# rename new_cases to data (a dataframe that contains information for all features to be prioritized)
data=new_cases #pd.read_csv("data_for_vaccine_plan_for_surging_83.csv")
# data.to_csv("data_for_vaccine_plan_for_surging_83.csv")

fits=pd.read_csv("curve_fitting_results_83_before_vaccine.csv")

In [17]:
# baseling projection result 546 + 92 days
with open('vaccine_scenarios_83/meta_base_data_0.pkl','rb') as f:
    meta=pickle.load(f)
    

In [10]:
us_pop_scale=3.282803*10**2 # US population/1M

In [51]:
# get projection summary from meta fits
cum_case_projection={'US':0}
cum_p_projection={'US':0}
states=0
for i in range(len(meta)):
    p=(meta[i]['data']['cum_case'].to_numpy()[-1])-(meta[i]['data']['cum_case'].to_numpy()[-93])
    case=p*meta[i]['actual_pop_scale'] 
    
    if meta[i]['region'].startswith('US.'):
        states+=1
        cum_case_projection['US']+=case
        cum_p_projection['US']+=p*meta[i]['actual_pop_scale']/us_pop_scale
    else:
        cum_case_projection[meta[i]['region']]=case
        cum_p_projection[meta[i]['region']]=p

In [53]:
# add features to data

for i,row in data.iterrows():
    r=row.location
    if r not in cum_case_projection: continue
    data.loc[i,'case_in_next_3months']=cum_case_projection[r]
    data.loc[i,'prevalence_in_next_3months']=cum_p_projection[r]


In [55]:
data

,location,population,cases,prevalence_per_million,vaccine_self_546,vaccine_cum_fraction_516,vaccine_cum_fraction_546,case_in_next_3months,prevalence_in_next_3months
0,Afghanistan,3.892834e+07,1713.333333,44.012493,4734.532258,0.007621,0.011269,91053.389599,2339.000000
1,Albania,2.877800e+06,3.533333,1.227790,2956.354839,0.131937,0.162756,71.945000,25.000000
2,Algeria,4.385104e+07,365.800000,8.341877,39112.903226,0.000855,0.027614,139665.571955,3185.000000
3,Angola,3.286627e+07,126.133333,3.837775,10134.080645,0.013083,0.022333,25044.096216,762.000000
4,Antigua and Barbuda,9.792800e+04,0.000000,0.000000,317.758065,0.132453,0.229797,7.246672,74.000000
...,...,...,...,...,...,...,...,...,...
143,Uzbekistan,3.346920e+07,425.133333,12.702226,30624.161290,0.024541,0.051991,205534.351059,6141.000000
144,Venezuela,2.843594e+07,1184.733333,41.663234,18564.322581,0.005556,0.025142,214606.061821,7547.000000
145,Zambia,1.838396e+07,2594.800000,141.144811,104.209677,0.003988,0.004158,94144.238676,5121.000000
146,Zimbabwe,1.486293e+07,563.733333,37.928823,5168.790323,0.033396,0.043829,120478.886262,8106.000000


## distribute vaccine by feature

### choose feature here

In [339]:
#feature='prevalence_per_million'
feature='cases'
#feature='case_in_next_3months'
#feature='prevalence_in_next_3months'

days=92 # days in projection 

In [340]:
# rank countries by features
# limit to top 35 countries 
countries=data.sort_values(by=[feature],ascending=False).head(35).copy()
countries.reset_index(inplace=True)
countries.drop(columns=['index'],inplace=True)

# calculate current vacine supply/feature
# rank by vac/feature
countries['vac/feature']=countries.vaccine_self_546/countries[feature]
countries_vac_sort=countries.sort_values(by=['vac/feature'],ascending=True).copy()
countries_vac_sort.reset_index(inplace=True)
countries_vac_sort.drop(columns=['index'],inplace=True)

In [341]:
#countries

In [28]:
#countries_vac_sort

- distribute donation until we have the same vac/feature for the most number of countries

In [343]:
plan=pd.DataFrame(columns=countries.columns)
# total donation from vaccine_X%_compute.ipynb
vac_aid_tot=5000247 #5*10**6


while vac_aid_tot>0:    
    
    # calculate which country to stop aiding
    feature_arr=countries_vac_sort[feature].to_numpy()
    vac_self=countries_vac_sort.vaccine_self_546.to_numpy()
    vac_case_ratio=countries_vac_sort['vac/feature'].to_numpy()
    regions=countries_vac_sort.location.to_numpy()
    print('n_aid; new_ratio; ratio of the next coutry')
    #n_aid=12
    ans=[]
    for n_aid in range(0,countries_vac_sort.shape[0]):
        # calculate new vac/feature ratio for countries[:n_aid]
        new_ratio=(vac_self[:n_aid].sum()+vac_aid_tot)/feature_arr[:n_aid].sum()
        ans.append([n_aid,regions[n_aid], new_ratio,vac_case_ratio[n_aid]])
        print([n_aid,regions[n_aid], new_ratio,vac_case_ratio[n_aid]])
    
    break_middle=False
    
    # stop the donation at the country where its vac/feature > new vac/feature ratio 
    if len(ans)>2:
        for i in range(0,len(ans)-1):
            if ans[i+1][2]<ans[i][3]:
                break_middle=True
                break
        print(len(ans),i)        
        if not break_middle:
            #i=i+1
            n_aid=ans[i][0]+1
        else:n_aid=ans[i][0]
    
    elif len(ans)==1:
            n_aid=1
    elif len(ans)==0:break
    else:
        if ans[1][2]<ans[0][3]:
            n_aid=1
        else: n_aid=2
        

    print('cut off ',n_aid)
    print()
    # apply donation
    #n_aid=ans[i][0]
    vac_case_ratio=(vac_self[:n_aid].sum()+vac_aid_tot)/feature_arr[:n_aid].sum()
    vac_aid=np.zeros_like(feature_arr)
    for i in range(n_aid):
        vac_aid[i]=max(vac_case_ratio*feature_arr[i]-vac_self[i],0)
    
    # check if aiding will lead to overflow (>100% coverage in 3 months)
    countries_vac_sort['vac_aid']=vac_aid
    countries_vac_sort['tot_vac']=countries_vac_sort.vac_aid+countries_vac_sort.vaccine_self_546

    countries_vac_sort['vaccine_cum_fraction_after_'+str(days)]=countries_vac_sort.vaccine_cum_fraction_546+\
                                        countries_vac_sort.tot_vac*days/countries_vac_sort.population
    countries_vac_sort['dosage_needed_to_vac_pop']=(1-countries_vac_sort.vaccine_cum_fraction_546) \
                                                    *countries_vac_sort.population/days-countries_vac_sort.vaccine_self_546
    
    countries_vac_sort['dosage_needed_to_vac_pop']=countries_vac_sort['dosage_needed_to_vac_pop'].clip(lower=0)
    
    # overflow countries only receive the amount of vaccine to vaccinate the entire population
    temp=countries_vac_sort[countries_vac_sort['dosage_needed_to_vac_pop']<=countries_vac_sort['tot_vac']]
    if temp.shape[0]==0:
        print('break')
        break
    
    if temp['dosage_needed_to_vac_pop'].sum()<vac_aid_tot:
        temp['vac_aid']=temp['dosage_needed_to_vac_pop']
    else:
        temp['vac_aid']=vac_aid_tot
    temp['tot_vac']=temp['vaccine_self_546']+temp['vac_aid']
    temp.drop(columns=['dosage_needed_to_vac_pop'],inplace=True)
    temp['vaccine_cum_fraction_after_'+str(days)]=temp.vaccine_cum_fraction_546+temp.tot_vac*days/temp.population
    temp['vac/feature_new']= temp['tot_vac']/temp[feature]
    plan=plan.append(temp,ignore_index=True)
    #print(temp.location)
    
    # deduct total aids from overflow country, prep for next iteration
    countries_vac_sort=countries_vac_sort[countries_vac_sort['dosage_needed_to_vac_pop']>countries_vac_sort['tot_vac']]
    countries_vac_sort.reset_index(inplace=True)
    countries_vac_sort.drop(columns=['index'],inplace=True)
    vac_aid_tot-=temp.vac_aid.sum()
    print(vac_aid_tot)
    #break
    
plan=plan.append(countries_vac_sort,ignore_index=True)
plan['vac/feature_new']=plan['tot_vac']/plan[feature]

n_aid; new_ratio; ratio of the next coutry
[0, 'Cuba', inf, 0.0]
[1, 'Zambia', 2589.3704688262937, 0.04016096709548784]
[2, 'Bangladesh', 1104.8383830010894, 0.6511763207507929]
[3, 'Iraq', 527.2406242182594, 0.6830820203627687]
[4, 'South Africa', 339.43439486450086, 2.472575946361348]
[5, 'Afghanistan', 178.69324853303465, 2.7633456759137958]
[6, 'Paraguay', 168.61903321331542, 3.5583891970437826]
[7, 'Iran', 158.89119954582387, 4.063890535803415]
[8, 'Mongolia', 121.2151533672584, 4.411714527456823]
[9, 'Colombia', 115.0894427094687, 4.60048538878028]
[10, 'Tunisia', 71.37438272659105, 4.94566477678945]
[11, 'Nepal', 68.82323116960895, 5.0991695641283]
[12, 'Oman', 67.38686103790424, 6.173068760611644]
[13, 'Argentina', 65.85302642085074, 6.743206619340729]
[14, 'Brazil', 53.96992060798757, 6.815648339219586]
[15, 'Bolivia', 34.462918687335346, 7.753100301236302]
[16, 'Russia', 34.16911923625228, 10.235837364305127]
[17, 'Uruguay', 31.935088914600584, 10.28951789756757]
[18, 'Costa 

In [344]:
feature

'cases'

In [345]:
vac_aid_tot

2461699.1318161716

In [346]:
plan.vac_aid.sum()

5000247.000000001

In [347]:
# vaccine plan
# vac_aid is the donation
# tot_vac/actual_pop_scale (tot_vac per million) to be input to projection sim 
plan

,location,population,cases,prevalence_per_million,vaccine_self_546,vaccine_cum_fraction_516,vaccine_cum_fraction_546,case_in_next_3months,prevalence_in_next_3months,vac/feature,vac_aid,tot_vac,vaccine_cum_fraction_after_92,vac/feature_new,dosage_needed_to_vac_pop
0,Mongolia,3.278292e+06,2325.666667,709.414130,1.026018e+04,0.469215,0.563107,2.021067e+04,6165.000000,4.411715,5.307898e+03,1.556808e+04,1.000000,6.694027,NaN
1,Colombia,4.825813e+07,29031.600000,601.589776,1.335595e+05,0.099574,0.182602,8.327906e+05,17257.000000,4.600485,2.952023e+05,4.287618e+05,1.000000,14.768796,NaN
2,Oman,5.106622e+06,2006.400000,392.901609,1.238565e+04,0.028962,0.101724,1.259497e+05,24664.000000,6.173069,3.747472e+04,4.986037e+04,1.000000,24.850663,NaN
3,Argentina,4.519578e+07,20147.933333,445.792387,1.358617e+05,0.133049,0.223231,2.789483e+05,6172.000000,6.743207,2.457327e+05,3.815943e+05,1.000000,18.939627,NaN
4,Brazil,2.101471e+08,70712.866667,336.492192,4.819540e+05,0.148240,0.217042,9.935756e+05,4728.000000,6.815648,1.306484e+06,1.788439e+06,1.000000,25.291557,NaN
5,Uruguay,3.473727e+06,1824.000000,525.084441,1.876808e+04,0.398073,0.560159,3.518885e+03,1013.000000,10.289518,0.000000e+00,1.876808e+04,1.057222,10.289518,NaN
6,Costa Rica,5.094114e+06,1503.800000,295.203445,1.584811e+04,0.142422,0.235754,3.317796e+04,6513.000000,10.538711,2.646883e+04,4.231694e+04,1.000000,28.140007,NaN
7,Kuwait,4.270563e+06,1688.333333,395.342097,2.064516e+04,0.200938,0.345966,1.559610e+04,3652.000000,12.228131,9.714526e+03,3.035969e+04,1.000000,17.982046,NaN
8,Chile,1.757678e+07,4740.733333,269.715705,6.824300e+04,0.523133,0.639610,1.133702e+04,645.000000,14.395030,6.102460e+02,6.885325e+04,1.000000,14.523754,NaN
9,United Kingdom,6.697602e+07,13481.866667,201.293951,2.127222e+05,0.485647,0.580930,4.956225e+03,74.000000,15.778394,9.236066e+04,3.050829e+05,1.000000,22.629127,NaN


In [348]:
feature

'cases'

In [349]:
plan.sort_values(by='vac/feature',ascending=True,inplace=True)

In [350]:
# save plan 
plan.to_csv("vaccine_plan_priority_"+feature+"_83.csv",index=False)